https://coolshell.cn/articles/11265.html

# Hello World

In [30]:
def hello(fn):
    def wrapper():
        print("hello, %s" % fn.__name__)
        fn()
        print("goodby, %s" % fn.__name__)
    return wrapper

@hello
def foo():
    print("i am foo")

foo()
print(foo.__name__) #输出 foo
print(foo.__doc__)  #输出 foo help doc

hello, foo
i am foo
goodby, foo
wrapper
None


输出:

```
hello, foo
i am foo
goodby, foo
```

# Decorator本质

当用@decorator来修饰函数func时，如下所示:
```python
@decorator
def func():
    pass
```
解释器会解释成如下语句：
```python
func = decorator(func)
```
把一个函数当参数传到另一个函数中，然后再回调。

但还有一个赋值，把decorator这个函数的返回值赋值回了原来的func。

函数式编程中，函数是一等公民，一切皆函数，把函数当成变量来使用。所以，decorator必需返回了一个函数出来给func，这就是所谓的higher order function 高阶函数，否则，后面当func()调用的时候就会出错。 

hello(foo)返回了wrapper()函数，所以，foo变成了wrapper的一个变量。而后面的foo()执行其实变成了wrapper()。

知道这点本质，当看到有多个decorator或是带参数的decorator，就不会懵逼。
- 多个decorator
```python
@decorator_one
@decorator_two
def func():
    pass
```
相当于：
```python
func = decorator_one(decorator_two(func))
```
每个decorator叠加调用func。
- 带参数的decorator：
```python
@decorator(arg1, arg2)
def func():
    pass
```
相当于：
```python
func = decorator(arg1,arg2)(func)
```
decorator(arg1, arg2)函数需要返回一个“真正的decorator”。

# 带参数及多个Decrorator

In [16]:
# hello = makeHtmlTag(arg1, arg2)(hello)
def makeHtmlTag(tag, *args, **kwds): # 这里是参数
    def real_decorator(fn): # 这里是fn
        # 拼接css
        css_class = "class='{0}'".format(kwds["css_class"]) if "css_class" in kwds else ""
        def wrapped(*args, **kwds):
            return '<{0} {1}>{2}</{0}>'.format(tag, css_class, fn(*args, **kwds))
        return wrapped
    return real_decorator

@makeHtmlTag(tag="x", css_class="bold_css")
@makeHtmlTag(tag="y", css_class="italic_css")
@makeHtmlTag(tag="z", css_class="italic_css")
def hello():
    return("hello world")

o = hello()
print(o)

<x class='bold_css'><y class='italic_css'><z class='italic_css'>hello world</z></y></x>


makeHtmlTag有两个参数。所以，为了让 hello = makeHtmlTag(arg1, arg2)(hello) 成功，makeHtmlTag必需返回一个decorator。

这就是为什么在makeHtmlTag中加入real_decorator()，这样，就可以进入到 decorator 的逻辑中—— decorator得返回一个wrapper，wrapper里回调hello。

In [ ]:
# class式的 Decorator

首先，先得说一下，decorator的class方式，还是看个示例：

In [19]:
class myDecorator(object):
 
    def __init__(self, fn):
        print("inside myDecorator.__init__()")
        self.fn = fn
 
    def __call__(self):
        self.fn()
        print("inside myDecorator.__call__()")

In [20]:
@myDecorator
def aFunc():
    print("inside aFunction()")

aFunc()

inside myDecorator.__init__()
inside aFunction()
inside myDecorator.__call__()


用类的方式声明一个decorator：
1. __init__，该方法是在给某个函数decorator时被调用，所以，需要有一个fn的参数，也就是被decorator的函数。
2. __call__，这个方法是在我们调用被decorator函数时被调用的。
上面输出可以看到整个程序的执行顺序。

看上去比“函数式”的方式更易读一些。

下面，我们来看看用类的方式来重写上面的html.py的代码：

In [27]:
class makeHtmlTagClass(object):
 
    def __init__(self, tag, css_class=""):
        self._tag = tag
        self._css_class = "class='{0}'".format(css_class) if css_class !="" else ""
 
    def __call__(self, fn):
        def wrapped(*args, **kwargs):
            return '<{0} {1}>{2}</{0}>'.format(self._tag, self._css_class, fn(*args, **kwargs))
        return wrapped

In [28]:
@makeHtmlTag(tag="a", css_class="bold_css")
@makeHtmlTag(tag="b", css_class="italic_css")
@makeHtmlTag(tag="c", css_class="italic_css")
def hello(name):
    return "Hello, {}".format(name)
 
print(hello("Hao Chen"))

<a class='bold_css'><b class='italic_css'><c class='italic_css'>Hello, Hao Chen</c></b></a>


注意：
1. 若decorator有参数，__init__成员就不能传入fn，而是在__call__时传入。
2. 此外，wrapped(*args, **kwargs) 这种方式来传递被decorator函数的参数。其中：args是一个参数列表，kwargs是参数dict，具体的细节，请参考Python的文档或是StackOverflow的这个问题，这里就不展开了

# 用Decorator为函数传递参数

三种方法：

- 第一种，约定好参数，直接修改参数

In [27]:
def decorate_B(function):
    def wrap_function(*args, **kwargs):
        str = 'Hello!'
        return function(str, *args, **kwargs)
    return wrap_function
 
@decorate_B
def print_message_B(str, *args, **kwargs):
    print(str)
 
print_message_B()

Hello!


- 第二种，通过 **kwargs，该方法会在kwargs中注入参数。

In [26]:
def decorate_A(function):
    def wrap_function(*args, **kwargs):
        kwargs['str'] = 'Hello!'
        return function(*args, **kwargs)
    return wrap_function
 
@decorate_A
def print_message_A(*args, **kwargs):
    print(kwargs['str'])
 
print_message_A()

Hello!


- 第三种，通过 *args 注入

In [28]:
def decorate_C(function):
    def wrap_function(*args, **kwargs):
        str = 'Hello!'
        #args.insert(1, str)
        args = args +(str,)
        return function(*args, **kwargs)
    return wrap_function
 
class Printer:
    @decorate_C
    def print_message(self, str, *args, **kwargs):
        print(str)
 
p = Printer()
p.print_message()

Hello!


# Decorator的副作用

被decorator的函数其实已经是另外一个函数了，对于最前面那个hello.py的例子来说，如果你查询一下foo.__name__的话，你会发现其输出的是“wrapper”，而不是我们期望的“foo”，这会给我们的程序埋一些坑。所以，Python的functool包中提供了一个叫wrap的decorator来消除这样的副作用。下面是我们新版本的hello.py。

In [29]:
from functools import wraps
def hello(fn):
    @wraps(fn)
    def wrapper():
        print("hello, %s" % fn.__name__)
        fn()
        print("goodby, %s" % fn.__name__)
    return wrapper
 
@hello
def foo():
    '''foo help doc'''
    print("i am foo")
    pass
 
foo()
print(foo.__name__) #输出 foo
print(foo.__doc__)  #输出 foo help doc

hello, foo
i am foo
goodby, foo
foo
foo help doc


当然，即使是你用了functools的wraps，也不能完全消除这样的副作用。

In [36]:
from inspect import getmembers, getargspec
from functools import wraps
 
def wraps_decorator(f):
    @wraps(f)
    def wraps_wrapper(*args, **kwargs):
        return f(*args, **kwargs)
    return wraps_wrapper
 
class SomeClass(object):
    @wraps_decorator
    def method(self, x, y):
        pass
 
obj = SomeClass()
for name, func in getmembers(obj, predicate=inspect.ismethod):
    print("Member Name: %s" % name)
    #print("Func Name: %s" % func.func_name)
    print("Args: %s" % getargspec(func)[0])
 
# 输出：
# Member Name: method
# Func Name: method
# Args: []

Member Name: method
Args: []


/Users/liuning11/anaconda3/envs/lr/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()


你会发现，即使是你你用了functools的wraps，你在用getargspec时，参数也不见了。

要修正这一问，我们还得用Python的反射来解决，下面是相关的代码：

In [37]:
def get_true_argspec(method):
    argspec = inspect.getargspec(method)
    args = argspec[0]
    if args and args[0] == 'self':
        return argspec
    if hasattr(method, '__func__'):
        method = method.__func__
    if not hasattr(method, 'func_closure') or method.func_closure is None:
        raise Exception("No closure for method.")
 
    method = method.func_closure[0].cell_contents
    return get_true_argspec(method)

当然，我相信大多数人的程序都不会去getargspec。所以，用functools的wraps应该够用了。

# 一些decorator的示例

## 给函数调用做缓存

这个示例实在是太经典了。

In [40]:
from functools import wraps
def memo(fn):
    cache = {}
    miss = object()
 
    @wraps(fn)
    def wrapper(*args):
        result = cache.get(args, miss)
        if result is miss:
            result = fn(*args)
            cache[args] = result
        return result
 
    return wrapper
 
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n - 1) + fib(n - 2)


fib(10)

55

这个斐波拉契数例的递归算法是相当没有效率的，因为会重复调用。比如：我们要计算fib(5)，于是其分解成fib(4) + fib(3)，而fib(4)分解成fib(3)+fib(2)，fib(3)又分解成fib(2)+fib(1)…… 你可看到，基本上来说，fib(3), fib(2), fib(1)在整个递归过程中被调用了两次。

而我们用decorator，在调用函数前查询一下缓存，如果没有才调用了，有了就从缓存中返回值。一下子，这个递归从二叉树式的递归成了线性的递归。

## Profiler的例子

这个例子很实用。

In [49]:
import cProfile, pstats
from io import StringIO
 
def profiler(func):
    def wrapper(*args, **kwargs):
        datafn = func.__name__ + ".profile" # Name the data file
        prof = cProfile.Profile()
        retval = prof.runcall(func, *args, **kwargs)
        #prof.dump_stats(datafn)
        s = StringIO.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(prof, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval
 
    return wrapper

## 注册回调函数

下面这个示例展示了通过URL的路由来调用相关注册的函数示例：

In [51]:
class MyApp():
    def __init__(self):
        self.func_map = {}
 
    def register(self, name):
        def func_wrapper(func):
            self.func_map[name] = func
            return func
        return func_wrapper
 
    def call_method(self, name=None):
        func = self.func_map.get(name, None)
        if func is None:
            raise Exception("No function registered against - " + str(name))
        return func()
 
app = MyApp()
 
@app.register('/')
def main_page_func():
    return "This is the main page."
 
@app.register('/next_page')
def next_page_func():
    return "This is the next page."
 
print(app.call_method('/'))
print(app.call_method('/next_page'))

This is the main page.
This is the next page.


注意：
1. 上面这个示例中，用类的实例来做decorator。
2. decorator类中没有__call__()，但是wrapper返回了原函数。所以，原函数没有发生任何变化。

## 给函数打日志

下面这个示例演示了一个logger的decorator，这个decorator输出了函数名，参数，返回值，和运行时间。

In [56]:
from functools import wraps
import time

def logger(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        ts = time.time()
        result = fn(*args, **kwargs)
        te = time.time()
        print("function      = {0}".format(fn.__name__))
        print("    arguments = {0} {1}".format(args, kwargs))
        print("    return    = {0}".format(result))
        print("    time      = %.6f sec" % (te-ts))
        return result
    return wrapper
 
@logger
def multipy(x, y):
    return x * y
 
@logger
def sum_num(n):
    s = 0
    for i in range(n+1):
        s += i
    return s
 
print(multipy(2, 10))
print(sum_num(100))
print(sum_num(10000000))

function      = multipy
    arguments = (2, 10) {}
    return    = 20
    time      = 0.000001 sec
20
function      = sum_num
    arguments = (100,) {}
    return    = 5050
    time      = 0.000009 sec
5050
function      = sum_num
    arguments = (10000000,) {}
    return    = 50000005000000
    time      = 0.581515 sec
50000005000000


下面看一个带log level参数的：

In [67]:
import inspect

def get_line_number():
    return inspect.currentframe().f_back.f_back.f_lineno
 
def logger(loglevel):
    def log_decorator(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            ts = time.time()
            result = fn(*args, **kwargs)
            te = time.time()
            print("function   = " + fn.__name__)
            print("    arguments = {0} {1}".format(args, kwargs))
            print("    return    = {0}".format(result))
            print("    time      = %.6f sec" % (te-ts))
            if (loglevel == 'debug'):
                print("    called_from_line : " + str(get_line_number()))
            return result
        return wrapper
    return log_decorator

@logger('debug')
def multipy(x, y):
    return x * y
 
@logger('debug')
def sum_num(n):
    s = 0
    for i in range(n+1):
        s += i
    return s
 
print(multipy(2, 10))
print(sum_num(100))

function   = multipy
    arguments = (2, 10) {}
    return    = 20
    time      = 0.000001 sec
    called_from_line : 34
20
function   = sum_num
    arguments = (100,) {}
    return    = 5050
    time      = 0.000008 sec
    called_from_line : 35
5050


但是，上面这个带log level参数的有两具不好的地方，
1. loglevel不是debug的时候，还是要计算函数调用的时间。
2. 不同level的要写在一起，不易读。
我们再接着改进：

In [66]:
import inspect
 
def advance_logger(loglevel):
 
    def get_line_number():
        return inspect.currentframe().f_back.f_back.f_lineno
 
    def _basic_log(fn, result, *args, **kwargs):
        print("function   = " + fn.__name__)
        print("    arguments = {0} {1}".format(args, kwargs))
        print("    return    = {0}".format(result))
 
    def info_log_decorator(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            result = fn(*args, **kwargs)
            _basic_log(fn, result, args, kwargs)
        return wrapper
 
    def debug_log_decorator(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            ts = time.time()
            result = fn(*args, **kwargs)
            te = time.time()
            _basic_log(fn, result, args, kwargs)
            print("    time      = %.6f sec" % (te-ts))
            print("    called_from_line : " + str(get_line_number()))
        return wrapper
 
    if loglevel is "debug":
        return debug_log_decorator
    else:
        return info_log_decorator
    
@advance_logger('debug')
def multipy(x, y):
    return x * y
 
@advance_logger('')
def sum_num(n):
    s = 0
    for i in range(n+1):
        s += i
    return s
 
print(multipy(2, 10))
print(sum_num(100))

function   = multipy
    arguments = ((2, 10), {}) {}
    return    = 20
    time      = 0.000001 sec
    called_from_line : 47
None
function   = sum_num
    arguments = ((100,), {}) {}
    return    = 5050
None


你可以看到两点，
1. 我们分了两个log level，一个是info的，一个是debug的，然后我们在外尾根据不同的参数返回不同的decorator。
2. 我们把info和debug中的相同的代码抽到了一个叫_basic_log的函数里，DRY原则。

## 一个MySQL的Decorator

下面这个decorator是我在工作中用到的代码，我简化了一下，把DB连接池的代码去掉了，这样能简单点，方便阅读。

In [ ]:
import umysql
from functools import wraps
 
class Configuraion:
    def __init__(self, env):
        if env == "Prod":
            self.host    = "coolshell.cn"
            self.port    = 3306
            self.db      = "coolshell"
            self.user    = "coolshell"
            self.passwd  = "fuckgfw"
        elif env == "Test":
            self.host   = 'localhost'
            self.port   = 3300
            self.user   = 'coolshell'
            self.db     = 'coolshell'
            self.passwd = 'fuckgfw'
 
def mysql(sql):
 
    _conf = Configuraion(env="Prod")
 
    def on_sql_error(err):
        print err
        sys.exit(-1)
 
    def handle_sql_result(rs):
        if rs.rows > 0:
            fieldnames = [f[0] for f in rs.fields]
            return [dict(zip(fieldnames, r)) for r in rs.rows]
        else:
            return []
 
    def decorator(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            mysqlconn = umysql.Connection()
            mysqlconn.settimeout(5)
            mysqlconn.connect(_conf.host, _conf.port, _conf.user, \
                              _conf.passwd, _conf.db, True, 'utf8')
            try:
                rs = mysqlconn.query(sql, {})
            except umysql.Error as e:
                on_sql_error(e)
 
            data = handle_sql_result(rs)
            kwargs["data"] = data
            result = fn(*args, **kwargs)
            mysqlconn.close()
            return result
        return wrapper
 
    return decorator
 
@mysql(sql = "select * from coolshell" )
def get_coolshell(data):
    ... ...
    ... ..

## 线程异步

下面量个非常简单的异步执行的decorator，注意，异步处理并不简单，下面只是一个示例。

In [ ]:
from threading import Thread
from functools import wraps
 
def async(func):
    @wraps(func)
    def async_func(*args, **kwargs):
        func_hl = Thread(target = func, args = args, kwargs = kwargs)
        func_hl.start()
        return func_hl
 
    return async_func
 
if __name__ == '__main__':
    from time import sleep
 
    @async
    def print_somedata():
        print 'starting print_somedata'
        sleep(2)
        print 'print_somedata: 2 sec passed'
        sleep(2)
        print 'print_somedata: 2 sec passed'
        sleep(2)
        print 'finished print_somedata'
 
    def main():
        print_somedata()
        print 'back in main'
        print_somedata()
        print 'back in main'
 
    main()

## 其它

关于更多的示例，你可以参看： [Python Decorator Library](https://wiki.python.org/moin/PythonDecoratorLibrary)

关于Python Decroator的各种提案，可以参看：[Python Decorator Proposals](https://wiki.python.org/moin/PythonDecoratorProposals)

（全文完）